In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import random
import statistics
import importlib

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [2]:
df = pd.read_csv('../data/gen_data.csv')
display(df.head())

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,3,2,5,5,0.133333,10,4.341249,2.573368,3,1.525419,17.236311,12.544197,1.374047,25.982207,1.166119,1.791735,3.068671369713104_0.840665429226872_7.05614187...
1,2,4,5,5,0.066667,20,8.333750,5.409342,2,3.511142,15.998487,13.688880,1.168721,66.145209,3.343115,3.497089,1.2259105735659919_13.001723614157617_6.803253...
2,2,6,4,5,0.666667,10,13.057627,5.089587,7,1.983813,58.798809,52.875749,1.112018,63.122647,2.882559,6.831266,13.25375466725487_7.935862899417814_12.8000545...
3,4,4,4,5,0.666667,25,14.895530,9.934758,2,6.626110,50.811883,38.906422,1.306002,67.958459,11.431050,19.368025,30.797824543245817_10.190419004083243_17.16747...
4,4,6,4,5,0.666667,30,23.865822,9.481228,6,3.766628,81.309284,75.100436,1.082674,118.064384,24.503399,28.642632,26.779115344385218_26.102422123788358_28.94647...


In [3]:
methods.plot_plotly(data=df['actual_cost'], data_label='Actual Cost')
methods.plot_plotly(data=df['optimal_cost'], data_label='Optimal Cost')
methods.multi_plot_plotly(data=[df['optimal_cost'], df['actual_cost']], data_label=['Optimal Cost', 'Actual Cost'])
